In [2]:
import duckdb
import pandas as pd

In [3]:
#tabla paises 
pais = duckdb.sql('''SELECT name AS Nombre FROM enunciado_paises.csv''')

In [6]:
#tabla jugador
jugador = duckdb.sql('''SELECT player_api_id AS ID, player_name AS Nombre FROM enunciado_jugadores.csv''')

In [24]:
#tabla temporada, el id fue creado con pandas
temporada = duckdb.sql('''SELECT DISTINCT season AS Temporada FROM enunciado_partidos.csv''')
df_temporada = temporada.df()         # paso el objeto duckdb a df de pandas
df_temporada['ID'] = range(len(df_temporada),0,-1) # para cada valor de temporada en la nueva columna pongo ese id
df_temporada

,Temporada,Id_temporada
0,2009/2010,8
1,2012/2013,7
2,2011/2012,6
3,2015/2016,5
4,2013/2014,4
5,2008/2009,3
6,2014/2015,2
7,2010/2011,1


In [32]:
# tabla liga, EL ID ESTA MAL, debería ser uno de estos [1,  1729,  4769,  7809, 10257, 13274, 15722, 17642, 19694,21518, 24558]
# pero como saber cual es cual? YA SE, gracias al country_id del partido
liga = duckdb.sql('''
                  SELECT DISTINCT l.name AS Nombre_liga, p.name AS Nombre_pais, part.league_id AS ID
                  FROM enunciado_liga.csv AS l
                  INNER JOIN enunciado_paises.csv AS p ON l.country_id = p.id
                  INNER JOIN enunciado_partidos.csv AS part ON l.country_id = part.country_id
                  ''')

df_liga = liga.df()
df_liga.to_csv('ligassssss')


In [31]:
# Tabla Equipos
equipo = duckdb.sql('''
                    SELECT DISTINCT eq.team_api_id AS ID_equipo, eq.team_long_name AS Nombre, par.league_id AS ID_liga
                    FROM enunciado_equipos.csv AS eq
                    INNER JOIN enunciado_partidos.csv AS par ON eq.team_api_id = par.home_team_api_id
                    ''')

df_equipo = equipo.df()
#EXPLICACION: basicamente hice el join en base a la liga donde ese equipo jugó de local, podria pasar que no hay jugado de local?
# según lo que estuve viendo no, porque me unió a todos los equipos

